# Financial Data Sources & APIs

A time series is essentially a sequence of data points arranged in chronological order. It’s like a story told over time, where each data point represents a snapshot at a specific moment. Time series data is indexed in time order, and examples include financial data such as equity prices, commodity prices, and forex rates, all of which are observed at regular time intervals.

## Financial Data Preprocessing

The first step in any data analysis is to parse the raw data. This involves extracting the data from the source, cleaning it up, and addressing any missing values. Financial data can come in many forms, but Python makes it easy to read and manipulate this data using powerful packages.

In this session, we'll focus on how to retrieve and store time series data using some popular Python libraries. Specifically, we’ll work with end-of-day data, intraday data, and option chain data. Additionally, we’ll explore how to read time series data from traditional local sources like SQL databases.

**Import Libraries**

In [1]:
# Import market data module from quantmod
import pandas as pd
from quantmod.markets import getData, getTicker

## Stock Data Retrieval

In [ ]:
# Fetching data by specifying the number of periods
df1 = getData("SBIN.NS", period="5d")
df1

In [ ]:
# Fetch data by specifying start and end dates
df2 = getData("SBIN.NS", start_date="2025-01-01", end_date="2025-01-10")
df2

In [ ]:
# Fetch data for year to date (YTD)
df3 = getData("SBIN.NS", period="ytd")
df3

In [ ]:
# Fetch data for multiple stocks
stocklist = ["SBIN.NS", "ITC.NS", "INFY.NS", "RELIANCE.NS", "TITAN.NS"]
df4 = getData(stocklist, period="10d")["Close"]
df4

In [ ]:
# Fetch data for multiple stocks with multiple fields
df5 = getData(stocklist, start_date="2025-01-01", end_date="2025-01-10")

In [ ]:
# Fetch intraday data for the last five days
df6 = getData("SBIN.NS", period="5d", interval="5m")
df6

## Options Data Retrieval (Indian Example)

We will now fetch and analyze option chain data from the NSE using the quantmod library.

The following example demonstrates how to use the `quantmod.derivatives` module to fetch option chain data for a given symbol and expiry. We will also calculate the Put-Call ratio, identify the maximum pain strike price, and access individual option quotes.

In [7]:
# Import option module from quantmod
from quantmod.derivatives import OptionData

In [8]:
# Get option chain for specified expiry
expiration = "31-Jul-2025"
opt = OptionData("NIFTY", expiration)

In [ ]:
# Get maximum pain strike
opt.get_maximum_pain_strike

In [ ]:
# Get put call ratio
opt.get_put_call_ratio

In [ ]:
# Get synthetic future price
opt.get_synthetic_future_price(25300)

In [ ]:
# Get option quote
opt.get_option_quote(25300, 'CE', 'buy')

In [13]:
# Get call option data
calls = opt.get_call_option_data

In [ ]:
# Filter call options by strike price range
call_df = calls.query("strikePrice >= 25000 & strikePrice <= 25300") 
call_df.head()

## Options Data Retrieval (US Example)

We'll now retrieve option chain for SPY for July 2025 expiration from yahoo finance and filter the output to display the first seven columns.

In [ ]:
# Get Ticker object from quantmod
spy = getTicker('SPY')
spy.options[:5]

In [ ]:
# Get SPY option chain for July 31st expiration
# https://finance.yahoo.com/quote/SPY250731C00620000/
options = spy.option_chain('2025-07-31')
options

In [ ]:
# Filter calls for strike above 600
df7 = options.calls[(options.calls['strike']>610) & (options.calls['strike']<630)]

# Check the filtered output
df7.iloc[:,:7]

## Data Storage & Analysis

Let's create a utility function to store market data for future use. First, we will retrieve a list of exchange tickers from Wikipedia to fetch the data iteratively. Next, we will create a database and a table to store the retrieved data. We'll use sqlite3 for this exercise and build a few functions to simplify data access. 

Refer [utils.py](../utils.py) for the utility functions.

## Data Wrangling

In [15]:
# Imports functions from utils file
import sys
import os
sys.path.append(os.path.abspath(".."))

from utils import (
    create_table,
    fetch_and_store_all,
    query_stock
)

In [ ]:
# ⚠️ Run this only once to create the table schema
# create_table()

In [ ]:
# ⚠️ Run this only once to fetch data and populate the database
# fetch_and_store_all()

In [ ]:
# Query the stock data
df = query_stock("TITAN")
df.head()

In [ ]:
# Column filter
df.filter(["close"])  

In [ ]:
# Row/index filter
df.filter(like="2024-06-20", axis=0)  

In [ ]:
# Conditional query
df.query("close > 3500")  

In [ ]:
# Close > Open
df.query("close > open")  

In [ ]:
# Volatility filter
df.query("high - low > 150")  

In [ ]:
# Open equals low
df.query("open == low")  

In [ ]:
# Combined conditions
df.query("close > 3500 & high - low > 150")  

## Data Resampling

In [ ]:
# Weekly resample (last value of week)
df_weekly = df.resample("W").last()
df_weekly

In [ ]:
# Weekly resample ending on Thursday
df_weekly_thu = df.resample("W-FRI").last()
df_weekly_thu

In [ ]:
# Monthly resample (end of month)
df_monthly = df.resample("ME").last()
df_monthly

**Note:** Option and stock data are often available only through premium data providers that offer high-quality, reliable, and production-grade datasets. The Indian and US examples in this coursework use publicly available data from Yahoo Finance to demonstrate techniques for wrangling and analyzing market data using popular Python libraries. While suitable for educational and exploratory purposes, this data may include delays, omissions, or inconsistencies. You are encouraged to substitute these examples with datasets relevant to your own geography or market, especially when accuracy, completeness, or real-time access is critical.

---
[Kannan Singaravelu](https://www.linkedin.com/in/kannansi) | Refer [Quantmod](https://kannansingaravelu.com/quantmod/), [Pandas](https://pandas.pydata.org/docs/user_guide/index.html), [SQLite](https://www.sqlite.org/docs.html) and [SQLAlchemy](https://docs.sqlalchemy.org/en/20) for more information.